In [1]:
from runLSTM import buildRunLSTM
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Flatten

import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
import pickle
import pandas as pd

Using TensorFlow backend.
c:\users\analytics\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
with open("data/03_07_19_lstm_traindata_01_with_labeled_family.pkl", "rb") as input_file:
    generated_data = pickle.load(input_file)

list_tuples = [x for x in generated_data]

In [24]:
b = buildRunLSTM()
X, y, max_features, maxlen, labels, valid_chars = b.read_data(list_tuples, 1)

In [6]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
labels = encoder.fit_transform(labels)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


<bound method BaseEstimator.get_params of LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)>

In [22]:
from keras.utils import np_utils
dummy_y = np_utils.to_categorical(labels)

In [29]:
myset = set(labels)


{'qadars', 'dyre', 'ran', 'locky DGA', 'mirai (rolling window of domains)', 'padcrypt', 'benign', 'cryptowall - static observed list', 'geodo DGA', 'bamital (domains valid ~5 days)', 'shiotob/urlzone/bebloh DGA - not date seeded', 'tinba', 'conficker', 'necurs', 'qakbot (domains valid ~5 days)', 'corebot DGA', 'dircrypt - not date seeded', 'pandabanker (rolling window of domains)', 'chinad (rolling window of domains)', 'Post Tovar GOZ DGA', 'Volatile Cedar / Explosive (not date seeded)', 'unknowndropper (rolling window of domains)', 'gozi (rolling window of domains)', 'feodo', 'tofsee', 'kraken (mix of date and non-date seeded hostnames and domains)', 'murofet (varying date seeds)', 'banjori - not date seeded', 'ramnit - not date seeded', 'fobber_v1', 'necurs (domains valid ~4 days)', 'proslikefan', 'suppobox (-1 to +1 days - time seeded)', 'pykspa_v2_fake', 'unknownjs (rolling window of domains)', 'pushdo DGA', 'simda', 'ranbyus', 'emotet', 'shifu', 'tofsee (rolling window of domains)

In [5]:
scaler = preprocessing.MinMaxScaler()
X = scaler.fit_transform(X)
#test_data = scaler.transform(X_malicious)

In [7]:
train_data, test_data, y_train, y_test, _, label_test = train_test_split(X, y, labels, test_size=0.4)

In [9]:
maxlen

66

In [19]:
from keras.layers import Input, CuDNNLSTM, RepeatVector, Dropout
from keras.models import Model
from keras.layers import TimeDistributed

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(CuDNNLSTM(128))
model.add(Dropout(0.5))
model.add(Dense(98-1, activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()
final_data = model.fit(train_data, y_train, batch_size=128, epochs=5, validation_data=(test_data, y_test))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 66, 128)           5120      
_________________________________________________________________
cu_dnnlstm_8 (CuDNNLSTM)     (None, 128)               132096    
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 97)                12513     
Total params: 149,729
Trainable params: 149,729
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking target: expected dense_8 to have shape (1,) but got array with shape (98,)

In [ ]:
print("Training set (images) shape: {shape}".format(shape=train_data.shape))
print("Testing set (images) shape: {shape}".format(shape=test_data.shape))

In [ ]:
label_dict = {
 0: 'Normal',
 1: 'DGA',
}

In [ ]:
from sklearn.model_selection import train_test_split
train_X,valid_X,train_ground,valid_ground = train_test_split(train_data,
                                                             train_data,
                                                             test_size=0.4,
                                                             random_state=13)

In [ ]:
train_X = np.reshape(train_X, (len(train_X), maxlen, 1))
valid_X = np.reshape(valid_X, (len(valid_X), maxlen, 1))
train_ground = np.reshape(train_ground, (len(train_ground), maxlen, 1))
valid_ground = np.reshape(valid_ground, (len(valid_ground), maxlen, 1))



In [ ]:
train_X.shape

In [ ]:
def encoder(input_img):
    #encoder
    #input = 1 X 59 (wide and thin)

    encoded = LSTM(128, return_sequences=True)(input_img)
    encoded = LSTM(64, return_sequences=True)(input_img)
    encoded = LSTM(32, return_sequences=False)(input_img)


    return encoded

def decoder(encoded):    
    #decoder
    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(32, return_sequences=True)(decoded)
    decoded = LSTM(64, return_sequences=True)(decoded)
    decoded = LSTM(n_features, return_sequences=True)(decoded)
    
    
    return decoded

In [ ]:
from keras.layers import Input, LSTM, RepeatVector, Dropout
from keras.models import Model
from keras.layers import TimeDistributed

num_classes = 2
epochs = 25
batch_size = 128
timesteps = train_X.shape[1]
n_features = train_X.shape[2] 


#lstm_autoencoder = Sequential()
# Encoder
#lstm_autoencoder.add(LSTM(encoding_dim, input_shape=(timesteps, n_features), return_sequences=True))
#lstm_autoencoder.add(LSTM(hidden_dim, return_sequences=False))
#lstm_autoencoder.add(RepeatVector(timesteps))
# Decoder
#lstm_autoencoder.add(LSTM(hidden_dim, return_sequences=True))
#lstm_autoencoder.add(LSTM(hidden_dim, return_sequences=True))
#lstm_autoencoder.add(TimeDistributed(Dense(n_features)))

#lstm_autoencoder.summary()
#lstm_autoencoder.compile(loss='mse', optimizer='adam')
input_img = Input(shape=(timesteps, n_features))
autoencoder = Model(input_img, decoder(encoder(input_img)))
autoencoder.compile(loss='mean_squared_error', optimizer = 'adam')
autoencoder.summary()

In [ ]:
autoencoder_train = autoencoder.fit(train_X, train_ground, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_ground))

In [ ]:
autoencoder.layers

In [ ]:
autoencoder.save_weights('autoencoder.h5')

In [ ]:
def fc(enco):
    #flat = Flatten()(enco)
    den = Dense(128, activation='relu')(enco)
    out = Dense(num_classes, activation='sigmoid')(den)
    return out

In [ ]:
encode = encoder(input_img)
full_model = Model(input_img,fc(encode))

In [ ]:
full_model.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

In [ ]:
full_model.summary()

In [ ]:
for l1,l2 in zip(full_model.layers[:2],autoencoder.layers[0:2]):
    l1.set_weights(l2.get_weights())

In [ ]:
for layer in full_model.layers[0:2]:
    layer.trainable = False

In [ ]:
from keras.utils import to_categorical
# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(y_train)
test_Y_one_hot = to_categorical(y_test)

# Display the change for category label using one-hot encoding
print('Original label:', y_train[2])
print('After conversion to one-hot:', train_Y_one_hot[2])

In [ ]:
classify_train = full_model.fit(train_X[0:10000], train_label[0:10000], batch_size=64,epochs=10,verbose=1,validation_data=(valid_X[0:1000], valid_label[0:1000]))

In [ ]:
#set layers trainable to true
for layer in full_model.layers[0:2]:
    layer.trainable = True
    
#recompile
full_model.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

#retrain
classify_train = full_model.fit(train_X[0:10000], train_label[0:10000], batch_size=64,epochs=10,verbose=1,validation_data=(valid_X[0:10000], valid_label[0:10000]))

In [ ]:
train_X,valid_X,train_label,valid_label = train_test_split(train_data,train_Y_one_hot,test_size=0.2,random_state=13)
train_X = np.reshape(train_X, (len(train_X), maxlen, 1))
valid_X = np.reshape(valid_X, (len(valid_X), maxlen, 1))

In [ ]:
full_model.predict(valid_X[0:10])

In [ ]:
valid_label[0:10]